In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../Health_cleaned.csv')
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,mariage_group,age_group,marriage_group,child
0,22861040,1,2,0,2,1992-04-01,62,56,5,1,...,NaN,NaN,NaN,NaN,NaN,0,married,middle,together,.m
1,501709010,1,1,0,1,2004-06-15,54,52,.m,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together,.m
2,501409020,1,1,0,1,2004-06-15,51,47,.m,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together,.m
3,502578010,1,2,1,3,2004-06-15,53,54,.m,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together,.m
4,501404010,1,1,0,1,2004-06-15,55,53,.m,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together,.m


In [3]:
df['riwbegy'] = pd.to_datetime(df['riwbegy'])

In [4]:
def count_jobs(row):
    job1 = not (row['rjweeks'] == '.m' or row['rjweeks'] == '0')
    job2 = not (row['rjweek2'] == '.m' or row['rjweek2'] == '0')

    if job1 and job2:
        return 2
    elif job1 or job2:
        return 1
    return 0


df['n_jobs'] = df.apply(count_jobs, axis=1)

In [5]:
df.sort_values('index_wave', inplace=True)

In [6]:
cpi_by_year = df.groupby('index_wave')['cpi'].median()
cpi_by_year

index_wave
1     0.934777
2     0.967805
3     1.028097
4     1.068196
5     1.133913
6     1.028097
7     1.068196
8     1.133913
9     1.204051
10    1.245748
11    1.306293
Name: cpi, dtype: float64

In [7]:
income_columns = ['riearn', 'siearn', 'ripena', 'sipena', 'rgov', 'sgov', 'total_work_income',
        'total_pension_income', 'total_gov_income', 'total_income']

In [8]:
for el in df.iterrows():
    for col in income_columns:
        df.loc[el[0], col] *= el[1]['index_wave']

In [8]:
unique_ids = df['hhidpn'].unique()
unique_ids.shape

(31532,)

In [9]:
df.columns

Index(['hhidpn', 'rmstat', 'ragender', 'rahispan', 'raracem', 'riwbegy',
       'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe', 'rdiabe', 'rcancre',
       'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp',
       'rhspnit', 'oop_spend', 'riearn', 'ripena', 'siearn', 'sipena', 'rcovr',
       'rcovs', 'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten',
       'index_wave', 'insured_gov', 'uninsured', 'retired', 'collegeplus',
       'year', 'cpi', 'rgov', 'sgov', 'inter_year', 'year_of_birth',
       'total_work_income', 'total_pension_income', 'total_gov_income',
       'total_income', 'without_work', 'n_jobs', 'riearn_delta',
       'siearn_delta', 'ripena_delta', 'sipena_delta', 'rgov_delta',
       'sgov_delta', 'total_work_income_delta', 'total_pension_income_delta',
       'total_gov_income_delta', 'total_income_delta', 'broken',
       'mariage_group', 'age_group', 'marriage_group', 'child'],
      dtype='object')

In [10]:
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,mariage_group,age_group,marriage_group,child
0,22861040,1,2,0,2,1992-04-01,62,56,5,1,...,NaN,NaN,NaN,NaN,NaN,0,married,middle,together,.m
18878,83974040,1,2,0,1,1992-09-22,50,51,3,0,...,NaN,NaN,NaN,NaN,NaN,0,married,middle,together,.m
18877,36832030,1,1,0,1,1992-09-22,63,57,3,0,...,NaN,NaN,NaN,NaN,NaN,0,married,middle,together,.m
18876,15014010,1,2,0,1,1992-09-22,55,62,2,0,...,NaN,NaN,NaN,NaN,NaN,0,married,middle,together,.m
18875,83613010,7,2,0,1,1992-09-22,57,.m,2,0,...,NaN,NaN,NaN,NaN,NaN,0,dead spouse,middle,dead spouse,.m


In [11]:
df['rmstat'].value_counts()

1     96997
7     27626
5     14136
1     11073
8      5476
3      5272
4      2551
7      2390
5      1529
2      1146
6      1059
8       479
3       476
4       314
2       118
.m      114
6         5
Name: rmstat, dtype: int64

In [12]:
def count_deltas(rows):
    income_deltas = {}

    for col in income_columns:
        income_deltas[f'{col}_delta'] = [np.nan]

    indexes = rows.index

    for i, (_, row) in zip(range(len(indexes)), rows.iterrows()):
        if i == 0:
            continue
        prev_row = rows.loc[indexes[i - 1]]

        valid_interval = True
        current_year = row['year']
        if current_year - prev_row['year'] > 2:
            valid_interval = False

        for col in income_columns:
            if not valid_interval or prev_row[col] == 0 or row[col] == 0:
                income_deltas[f'{col}_delta'].append(np.nan)
            else:
                income_deltas[f'{col}_delta'].append(row[col] / prev_row[col] * 100  - 100)

    return income_deltas

In [13]:
def check_if_broken(rows):
    indexes = rows.index
    broken = False
    was_married = False

    statuses = rows['rmstat']

    for st in statuses:
        if st == '.m' or int(st) not in [1, 2, 5, 7]:
            broken = False
            break
            
        if int(st) == 1:
            was_married = True

        if int(st) in [2, 5, 7]:
            if was_married:
                broken = True
            else:
                broken = False
                break
    
        if broken and int(st) not in [2, 5, 7]:
            broken = False
            break

    return int(broken)

In [15]:
broken = []

for uid in unique_ids:
    result = check_if_broken(df[df['hhidpn'] == uid])

    df.loc[df['hhidpn'] == uid, 'broken'] = result

df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,mariage_group,age_group,marriage_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle,together
18878,15014010,1,2,0,1,1992-09-22,55,62,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle,together
18877,83613010,7,2,0,1,1992-09-22,57,.m,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,dead spouse,middle,dead spouse
18876,83977010,1,2,0,1,1992-09-22,55,57,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle,together
18875,77679030,1,2,0,2,1992-09-22,46,53,0,.m,...,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle,together


In [14]:
df[df['broken'] == True].groupby('hhidpn')['rmstat'].unique()

hhidpn
10004040     [1, 1, 7]
10075020     [1, 1, 7]
10109030        [1, 7]
10210020     [1, 1, 7]
10395020     [1, 1, 7]
               ...    
918529020       [1, 2]
919258010       [1, 5]
919258020       [1, 5]
920538010       [1, 2]
920538020       [1, 2]
Name: rmstat, Length: 3480, dtype: object

In [18]:
for uid in unique_ids:
    break
    income_deltas = count_deltas(df[df['hhidpn'] == uid])

    for col in income_columns:
        df.loc[df['hhidpn'] == uid, f'{col}_delta'] = income_deltas[f'{col}_delta']

In [19]:
df[df['hhidpn'] == 22861010][income_columns + [f'{col}_delta' for col in income_columns]]

,riearn,siearn,ripena,sipena,rgov,sgov,total_work_income,total_pension_income,total_gov_income,total_income,riearn_delta,siearn_delta,ripena_delta,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta
4,23485.564002,0.0,0.0,0.0,0.0000,3522.8347,23485.564002,0.0,3522.8347,27008.398702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11019,25280.287153,0.0,0.0,0.0,0.0000,3768.0933,25280.287153,0.0,3768.0933,29048.380453,-8.265564,NaN,NaN,NaN,NaN,-8.844943,-8.265564,NaN,-8.844943,-8.341135
25453,23157.354808,0.0,0.0,0.0,0.0000,5252.3936,23157.354808,0.0,5252.3936,28409.748408,-18.366358,NaN,NaN,NaN,NaN,24.221806,-18.366358,NaN,24.221806,-12.841913
38758,25473.632823,0.0,0.0,0.0,0.0000,10436.9110,25473.632823,0.0,10436.9110,35910.543823,2.827702,NaN,NaN,NaN,NaN,85.747472,2.827702,NaN,85.747472,18.157907
54929,9841.727323,0.0,0.0,0.0,8843.4951,12653.6490,9841.727323,0.0,21497.1441,31338.871423,-62.608905,NaN,NaN,NaN,NaN,17.336072,-62.608905,NaN,99.340952,-15.540382
74948,19852.633829,0.0,0.0,0.0,8338.1064,7279.2988,19852.633829,0.0,15617.4052,35470.039029,107.386738,NaN,NaN,NaN,-3.065657,-40.856375,107.386738,NaN,-25.310030,16.362352
97052,0.000000,0.0,0.0,0.0,8177.9585,7197.1133,0.000000,0.0,15375.0718,15375.071800,NaN,NaN,NaN,NaN,4.767992,5.613624,NaN,NaN,5.162142,-53.697266
103235,0.000000,0.0,0.0,0.0,8028.0000,29991.3520,0.000000,0.0,38019.3520,38019.352000,NaN,NaN,NaN,NaN,11.312073,372.517084,NaN,NaN,180.393130,180.393130
121846,0.000000,0.0,0.0,0.0,7955.8916,10566.4180,0.000000,0.0,18522.3096,18522.309600,NaN,NaN,NaN,NaN,19.323598,-57.579421,NaN,NaN,-41.340916,-41.340916
140706,0.000000,0.0,0.0,0.0,0.0000,8585.2588,0.000000,0.0,8585.2588,8585.258800,NaN,NaN,NaN,NaN,NaN,1.217546,NaN,NaN,-42.258448,-42.258448


In [15]:
def reformat_rhltc(el):
    if el == '.m':
        return el
    return 3 - int(el)


def return_group(el):
    if el == '.m':
        return '.m'
    if int(el) == 1 or int(el) == 3:
        return 'together'
    if int(el) == 2:
        return 'absent spouse'
    if int(el) == 4:
        return 'separated'
    if int(el) == 5:
        return 'divorced'
    if int(el) == 6:
        return '.m'
    if int(el) == 7:
        return 'dead spouse'
    if int(el) == 8:
        return 'never married'


def return_age_group(el):
    age = int(el)
    if age < 45:
        return 'young'
    elif 45 <= age <= 65:
        return 'middle'
    elif 65 <= age <= 80:
        return 'old'
    else:
        return 'dead outside'

df['rhltc'] = df['rhltc'].apply(reformat_rhltc)
df['marriage_group'] = df['rmstat'].apply(return_group)
df['age_group'] = df['ragey_b'].apply(return_age_group)

In [41]:
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,mariage_group,age_group,marriage_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle,together
90806,502578010,1,2,1,3,2004-06-15,53,54,.m,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together
90812,501404010,1,1,0,1,2004-06-15,55,53,.m,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together
90831,501699020,1,1,0,1,2004-06-15,52,48,.m,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,together
90832,501445010,8,2,0,2,2004-06-15,56,.m,.m,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,middle,never married


In [33]:
df['age_group'].value_counts().reindex(['young', 'middle', 'old', 'dead outside'])

young            32770
middle          110607
old              25655
dead outside      1729
Name: age_group, dtype: int64

In [27]:
# child_csv = pd.read_csv('../child.csv', low_memory=False)
# child_csv.shape

(103430, 3)

In [28]:
# child_csv['hhidpn'] = child_csv['hhidpn'].astype(int)

In [29]:
# child_csv = child_csv[child_csv['child'] != '.m']
# child_csv['child'] = child_csv['child'].astype(int)
# child_csv.sort_values(by='index_wave', inplace=True)
# child_csv.shape

(7054, 3)

In [35]:
# df['child'] = [np.nan] * df.shape[0]

In [57]:
# for id in unique_ids:
#     cid = id // 1000
#     tmp = child_csv[child_csv['hhidpn'] == cid]

#     for i, el in tmp.iterrows():
#         df.loc[(df['hhidpn'] == id) & (df['index_wave'] == el['index_wave']), 'child'] = el['child']

In [59]:
# df['child'].replace({np.nan: '.m'}, inplace=True)
# df['child'].value_counts()

.m     165177
0.0      3157
1.0      2427
Name: child, dtype: int64

In [16]:
df.to_csv('../Health_cleaned_income_delta.csv', index=False)